# Install libraries 


In [1]:
import sys, os, json
import mne, sklearn, wandb
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
from nilearn import datasets, image, masking, plotting
from nilearn.input_data import NiftiLabelsMasker


# animation part
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
# from celluloid import Camera   # it is convinient method to animate
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation


## torch libraries 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset

from pytorch_model_summary import summary


/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:
%load_ext autoreload
%autoreload 2
sys.path.insert(1, os.path.realpath(os.path.pardir))

from utils import get_datasets
from utils import preproc
from utils import torch_dataset
from utils import train_utils
from utils import inference
from utils.models_arch import autoencoder_v3

# Set all hyperparameters
- Cuda and GPU.
- Parameters of dataset. 
- random seed( if necessary). 


In [3]:
print(torch.cuda.is_available(), torch.cuda.device_count())
torch.cuda.set_device(3)

True 4


In [4]:
config = dict(  
                dataset_name = 'NODDI', # NODDI
                fps = 250,
                new_fps=250, 
                n_channels = 64, # 64 
                n_roi = 17,
                test_sec = 60, # in seconds.
                freqs = np.logspace(np.log10(1), np.log10(49), 16),
                
                
                bold_delay = 5,
                to_many = True,
                random_subsample = True,
                sample_per_epoch = 2048, 
                WINDOW_SIZE = 2048,
                    
                optimizer='adam',
                lr=0.0001,
                weight_decay=0, 
                batch_size=128, 

                loss_function = 'mse', 
                model_type = '1D_CNN_AE_changed'
                )


hp_autoencoder = dict(n_electrodes=config['n_channels'],
                      n_freqs = len(config['freqs']),
                      n_channels_out = config['n_roi'],
                      n_res_block=0, 
                      channels=[32, 32, 64, 64], 
                      kernel_sizes=[15, 11, 5],
                      strides=[8, 8, 4]
                     )


config = {**hp_autoencoder, **config}

params_train = {'batch_size': config['batch_size'],
                'shuffle': True,
                'num_workers': 0}

params_val = {'batch_size': config['batch_size'],
              'shuffle': False}

In [5]:
# import random

# torch.manual_seed(0)
# random.seed(0)  # python operation seed
# np.random.seed(0)

# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True


# Upload preprocessed dataset from np files. 
It should accelerate speed of experiments.

In [6]:
with open("../data/preproc/labels_roi_17.json", 'r') as f:
    labels_roi = json.load(f)


if config['dataset_name']=='CWL':
    dataset_path = '../data/preproc/CWL/trio1_CWL_50_hz_17_roi_1_49_freqs.npz'
    
elif config['dataset_name']=='NODDI':
    # dataset_path = '../data/preproc/NODDI/32_NODDI_50_hz_17_roi_1_49_freqs.npz'
    dataset_path = '../data/preproc/NODDI/32_NODDI_250_hz_17_roi_1_49_freqs.npz'

else:
    print('no such dataset')


# download data
data = np.load(dataset_path)

x_train = np.clip(data['x_train'], 0, np.max(data['x_train']))
x_test = np.clip(data['x_test'], 0, np.max(data['x_test']))

x_train = np.log(x_train + 1e-5)
x_test = np.log(x_test + 1e-5)
print(x_train.shape)
train_dataset_prep = (x_train, data['y_train'])
test_dataset_prep = (x_test, data['y_test'])



# apply time dealy corrected
train_dataset_prep = preproc.bold_time_delay_align(train_dataset_prep, 
                                                   config['new_fps'],
                                                   config['bold_delay'])
test_dataset_prep = preproc.bold_time_delay_align(test_dataset_prep, 
                                                  config['new_fps'],
                                                  config['bold_delay'])


print('Size of train dataset:', train_dataset_prep[0].shape, train_dataset_prep[1].shape)
print('Size of test dataset:', test_dataset_prep[0].shape, test_dataset_prep[1].shape)

# torch dataset creation 
torch_dataset_train = torch_dataset.CreateDataset_eeg_fmri(train_dataset_prep, 
                                                            random_sample=config['random_subsample'], 
                                                            sample_per_epoch=config['sample_per_epoch'], 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])

torch_dataset_test = torch_dataset.CreateDataset_eeg_fmri(test_dataset_prep, 
                                                            random_sample=False, 
                                                            sample_per_epoch=None, 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])


# init dataloaders for training
train_loader = torch.utils.data.DataLoader(torch_dataset_train, **params_train)
val_loader = torch.utils.data.DataLoader(torch_dataset_test, **params_val)

(64, 16, 146460)
Size of train dataset: (64, 16, 145210) (17, 145210)
Size of test dataset: (64, 16, 13750) (17, 13750)


# Init Model, Loss, optimizers

In [7]:
model = autoencoder_v3.AutoEncoder1D(**hp_autoencoder)

loss_func = train_utils.make_mse_loss()
train_step = train_utils.train_step

optimizer = optim.Adam(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])


print(summary(model, torch.zeros(4, config['n_channels'], 
                                 len(config['freqs']),
                                 config['WINDOW_SIZE']), show_input=False))


-------------------------------------------------------------------------
      Layer (type)          Output Shape         Param #     Tr. Param #
          Conv2d-1     [4, 16, 16, 2048]           1,040           1,040
          Conv1d-2         [4, 32, 2048]           8,224           8,224
         Block1D-3          [4, 32, 256]           5,536           5,536
         Block1D-4           [4, 64, 32]          19,264          19,264
         Block1D-5            [4, 64, 8]          21,312          21,312
         Block1D-6            [4, 64, 8]          21,312          21,312
          Conv1d-7            [4, 17, 8]           1,105           1,105
        Upsample-8         [4, 17, 2048]               0               0
Total params: 77,793
Trainable params: 77,793
Non-trainable params: 0
-------------------------------------------------------------------------


/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


# Model training

In [ ]:
n_runs = 3

for i in range(n_runs):
    
    model = autoencoder_v3.AutoEncoder1D(**hp_autoencoder)

    loss_func = train_utils.make_mse_loss()
    train_step = train_utils.train_step

    optimizer = optim.Adam(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])
    
    
    parameters = {
        'EPOCHS': 1000,
        'model': model, 
        'train_loader': train_loader, 
        'val_loader': val_loader, 
        'loss_function': loss_func,
        'train_step': train_step,
        'optimizer': optimizer, 
        'device': 'cuda', 
        'raw_test_data': test_dataset_prep,
        'show_info': 5, 
        'num_losses': 5,
        'labels': labels_roi,
        'inference_function': inference.model_inference_function, 
        'to_many': config['to_many']
    }



    path_to_save_wandb = 'common/koval_alvi/Checkpoints/wandb_brain'
    
    
    with wandb.init(project="eeg_fmri", config=config, save_code=True):
        
        wandb.define_metric("val/corr_mean", summary="max")

        if i == 0: 
            exp_name = wandb.run.name
        
        wandb.run.name = exp_name +'_run_' + str(i)
        
        print(config)
        print(parameters['model'])
        print(summary(model, torch.zeros(4, config['n_channels'],
                                         len(config['freqs']), config['WINDOW_SIZE']), show_input=False))
        
        model = train_utils.wanb_train_regression(**parameters)
        

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koval_alvi (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'n_electrodes': 64, 'n_freqs': 16, 'n_channels_out': 17, 'n_res_block': 0, 'channels': [32, 32, 64, 64], 'kernel_sizes': [15, 11, 5], 'strides': [8, 8, 4], 'dataset_name': 'NODDI', 'fps': 250, 'new_fps': 250, 'n_channels': 64, 'n_roi': 17, 'test_sec': 60, 'freqs': array([ 1.        ,  1.29622305,  1.68019419,  2.17790642,  2.8230525 ,
        3.65930571,  4.74327639,  6.14834418,  7.96962542, 10.33041213,
       13.39051828, 17.35709839, 22.49867095, 29.16329579, 37.8021361 ,
       49.        ]), 'bold_delay': 5, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 2048, 'WINDOW_SIZE': 2048, 'optimizer': 'adam', 'lr': 0.0001, 'weight_decay': 0, 'batch_size': 128, 'loss_function': 'mse', 'model_type': '1D_CNN_AE_changed'}
AutoEncoder1D(
  (spatial_reduce_2d): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
  (spatial_reduce): Conv1d(256, 32, kernel_size=(1,), stride=(1,))
  (downsample_blocks): ModuleList(
    (0): Block1D(
      (downsample): AvgPool1d(kernel_size=(8,), s

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


.................

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


...

...............
Epoch 220 train loss_0 : 0.404 val loss_0 : 1.36 train loss_1 : 0.743 val loss_1 : 0.0123 
.

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


................

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=linear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


...............

# 